# Examples: `bunch` 

[In progress]

In [ ]:
import importlib
import sys
sys.path.append('..')

from ipywidgets import interact
from ipywidgets import interactive
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
import psdist.bunch as psb
import psdist.image as psi
import psdist.plotting as psp
import seaborn as sns

In [ ]:
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['grid'] = False

In [ ]:
n = 4
k = 100000
X = np.zeros((1, n))

state = np.random.RandomState(1950)
_Sigma = np.identity(n)
for i in range(n):
    for j in range(i):
        _Sigma[i, j] = _Sigma[j, i] = state.uniform(-0.2, 0.2)

X = np.vstack([X, state.multivariate_normal(np.zeros(n), _Sigma, size=k)])
for _ in range(4):
    scale = state.uniform(0.5, 1.5, size=n)
    loc = state.uniform(-2.0, 2.0, size=n)
    X = np.vstack([X, state.normal(loc=loc, scale=scale, size=(int(0.25 * k), n))])
X = X - np.mean(X, axis=0)

## Corner plot 

In [ ]:
dims = ['x', 'xp', 'y', 'yp']
alpha = 0.5
cmaps = [pplt.Colormap(name, left=0.2) for name in ['blues', 'reds']]
diag_colors = [cmap(0.75) for cmap in cmaps]
state = np.random.RandomState(1243)
diag_kind = 'step'

axes = psp.corner(
    data=X, 
    bins='auto',
    kind='hist',
    diag_kind=diag_kind,
    limits=None,
    labels=dims,
    samples=None,
    diag_height_frac=0.6,
    autolim_kws=None,
    diag_kws=dict(color=diag_colors[0], alpha=alpha),
    fig_kws=None,
    prof=False,
    prof_kws=None,
    return_fig=False,
    return_mesh=False,
    cmap=cmaps[0],
    alpha=alpha,
    rms_ellipse=True,
    rms_ellipse_kws=dict(level=2.0, color=diag_colors[0]),
)
axes = psp.corner(
    data=(X + state.uniform(-2.0, 2.0, size=(1, X.shape[1]))), 
    axes=axes,
    diag_kind=diag_kind,
    diag_kws=dict(color=diag_colors[1], alpha=alpha),
    autolim_kws=dict(),
    cmap=cmaps[1],
    alpha=alpha,
    rms_ellipse=True,
    rms_ellipse_kws=dict(level=2.0, color=diag_colors[1]),
)

## Interactive slicing

In [ ]:
psp.interactive_proj2d_discrete(X)

## 2D plotting

In [ ]:
axis = (0, 3)
_dims = [dims[k] for k in axis]
_X = X[:, axis]

In [ ]:
image, centers = psb.histogram(_X, bins=35, centers=True)

fig, ax = pplt.subplots()
ax = psp.image(
    image, x=centers[0], y=centers[1], ax=ax, 
    divide_by_max=True,
    colorbar=True,
    rms_ellipse=True,
    rms_ellipse_kws=dict(level=(1.0, 2.0, 3.0), color='white', alpha=0.5),
    mask_zero=False,
)
ax.format(xlabel=_dims[0], ylabel=_dims[1])
plt.show()

In [ ]:
fig, ax = pplt.subplots()
psp.hist2d(_X, ax=ax, mask_zero=True, bins=50, cmap=pplt.Colormap('mono', left=0.1))

fractions = [0.10, 0.90, 1.00]
cmap = pplt.Colormap('flare')
colors = [cmap(i) for i in [0.0, 0.5, 1.0]]
colors = ['red2', 'red5', 'red9']
for fraction, color in zip(fractions, colors):
    level = psb.enclosing_ellipsoid(_X, fraction=fraction)
    ax = psp.rms_ellipse(
        Sigma=psb.cov(X), center=psb.mean(X), level=level, ax=ax,
        color=color, lw=1.5,
        label=f'{(100.0 * fraction)}',
    )
ax.legend(loc='r', ncols=1)